In [31]:
knn_model_file = "https://github.com/PacktPublishing/" + \
    "Amazon-SageMaker-Cookbook/raw/master/" + \
    "Chapter12/files/model.knn.tar.gz"

In [19]:
!mkdir -p tmp

In [20]:
!wget -O tmp/model.knn.tar.gz {knn_model_file}

--2021-05-01 12:40:22--  https://github.com/PacktPublishing/Amazon-SageMaker-Cookbook/raw/master/Chapter12/files/model.knn.tar.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Chapter12/files/model.knn.tar.gz [following]
--2021-05-01 12:40:22--  https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Chapter12/files/model.knn.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 764 [application/octet-stream]
Saving to: ‘tmp/model.knn.tar.gz’

tmp/model.knn.tar.g 100%[===================>]     764  --.-KB/s    in 0s  

In [21]:
s3_bucket = "sagemaker-cookbook-bucket"
prefix = "chapter12"

In [22]:
model_data = f"s3://{s3_bucket}/{prefix}/files/model.knn.tar.gz"

In [23]:
!aws s3 cp tmp/model.knn.tar.gz {model_data}

upload: tmp/model.knn.tar.gz to s3://sagemaker-cookbook-bucket/chapter12/files/model.knn.tar.gz


In [24]:
from sagemaker import Model
from sagemaker import KNNPredictor
from sagemaker.image_uris import retrieve

In [25]:
image_uri = retrieve("knn", region="us-east-1")
image_uri

'382416733822.dkr.ecr.us-east-1.amazonaws.com/knn:1'

In [26]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()

model = Model(model_data=model_data,
              image_uri=image_uri,
              role=role,
              predictor_cls=KNNPredictor)

In [27]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.t2.medium')

-------------------!

In [28]:
import numpy as np

test_data = [0.3713707, 0.3713707, 0.3713707, 
             0.89655172, 0.89655172]
test_data_np = np.array(test_data)
predictor.predict(test_data_np)

[label {
   key: "predicted_label"
   value {
     float64_tensor {
       values: 1.0
     }
   }
 }]

In [29]:
predictor.delete_endpoint()